<a href="https://colab.research.google.com/github/Me-RockingAbhi/My-Projects/blob/master/symptom_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
data=pd.read_excel("/content/gdrive/My Drive/projet12.xlsx")
data.shape

(1094, 13)

In [0]:
for i in data['Allergy']:
  print(i)

In [0]:
from sklearn.utils import shuffle
data=shuffle(data)
for i in data['Allergy']:
  print(i)

In [0]:
data.isnull().sum()
l=[]
for i in data['travel_history_to_country']:
  if(pd.isnull(i)==True):
    l.append('none')
  else:
    l.append(i)
data['travel_history_to_country']=l
l=[]
for i in data["date_of_visit"]:
  if(pd.isnull(i)==True):
    l.append('none')
  else:
    l.append(i)
data['date_of_visit']=l


In [0]:
data.info()
data['Disease']=data['Allergy']
data=data.drop('Allergy',axis=1)
data.head()

In [0]:
data['sleep_disruption']=data['disruption_of_sleep']
data=data.drop('disruption_of_sleep',axis=1)


In [0]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['genetics_factor']=le.fit_transform(data['genetics_factor'])
le=LabelEncoder()
data['blood_transfusion']=le.fit_transform(data['blood_transfusion'])
le=LabelEncoder()
data['Disease']=le.fit_transform(data['Disease'])
le=LabelEncoder()
data['sleep_disruption']=le.fit_transform(data['sleep_disruption'])
le=LabelEncoder()
data['travel_history_to_country']=le.fit_transform(data['travel_history_to_country'])


In [0]:
le=LabelEncoder()
data['gender']=le.fit_transform(data['gender'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 450 to 708
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   travel_history_to_country  1094 non-null   int64 
 1   date_of_visit              1094 non-null   object
 2   Symptoms                   1094 non-null   object
 3   past_history_of_diseases   1094 non-null   object
 4   genetics_factor            1094 non-null   int64 
 5   blood_presure              1094 non-null   object
 6   emotional_problems         1094 non-null   object
 7   change_in_skin_color       1094 non-null   object
 8   blood_transfusion          1094 non-null   int64 
 9   age                        1094 non-null   int64 
 10  gender                     1094 non-null   int64 
 11  Disease                    1094 non-null   int64 
 12  sleep_disruption           1094 non-null   int64 
dtypes: int64(7), object(6)
memory usage: 119.7+ KB


In [0]:
l=[]
for i in data['date_of_visit']:
  if(i!='none' and i!='None'):
    name=str(i).split('-')[0]
    l.append(name)
  elif(i=='None'):
    l.append('none')
  else:
    l.append(i)
data['date_of_visit']=l
le=LabelEncoder()
data['date_of_visit']=le.fit_transform(data['date_of_visit'])

In [0]:
ytrain=data['Disease']
data=data.drop('Disease',axis=1)

In [0]:
import nltk
nltk.download("stopwords")
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
def cleaned_data(symptoms):
  symptoms=symptoms.lower()
  tokenizer=RegexpTokenizer(r'\w+')
  token=tokenizer.tokenize(symptoms)
  stopword=set(stopwords.words('english'))
  new_token=[token for token in token if token not in stopword]
  ps=PorterStemmer()
  stem_token=[ps.stem(token)for token in new_token]
  clean_review=' '.join(stem_token)
  return(clean_review)
data['new_symp']=[cleaned_data(i) for i in data['Symptoms']]
data['new_past']=[cleaned_data(i) for i in data['past_history_of_diseases']]
data['new_emot']=[cleaned_data(i) for i in data['emotional_problems']]
data['new_skin']=[cleaned_data(i) for i in data['change_in_skin_color']]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
for i in data['new_past']:
  print(i)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vec = TfidfVectorizer(stop_words='english',max_features=500)
symp= tfidf_vec.fit_transform(data['new_symp'].values.tolist())
symp= pd.DataFrame(symp.toarray(), columns=tfidf_vec.get_feature_names())
tfidf_vec = TfidfVectorizer(stop_words='english',max_features=250)
past= tfidf_vec.fit_transform(data['new_past'].values.tolist())
past= pd.DataFrame(past.toarray(), columns=tfidf_vec.get_feature_names())
tfidf_vec = TfidfVectorizer(stop_words='english',max_features=250)
emot= tfidf_vec.fit_transform(data['new_emot'].values.tolist())
emot= pd.DataFrame(emot.toarray(), columns=tfidf_vec.get_feature_names())
tfidf_vec = TfidfVectorizer(stop_words='english',max_features=250)
skin= tfidf_vec.fit_transform(data['new_skin'].values.tolist())
skin= pd.DataFrame(skin.toarray(), columns=tfidf_vec.get_feature_names())


In [0]:
data= pd.concat([data,symp], axis=1)
data= pd.concat([data,past], axis=1)

In [0]:
data= pd.concat([data,emot], axis=1)
data= pd.concat([data,skin], axis=1)

In [0]:
l=[]
for i in data['blood_presure']:
  if(i=='normal' or i=='Normal'):
    l.append(2)
  elif(i=='low' or i=='Low'):
    l.append(1)
  elif(i=='high' or i=='High' or i=='hn'):
    l.append(3)
  else:
    name=str(i).split('/')[0]
    if(int(name)>130):
      l.append(3)
    elif(int(name)<120):
      l.append(1)
    else:
      l.append(2)
data['blood_presure']=l

In [0]:
data=data.drop('new_symp',axis=1)
data=data.drop('new_past',axis=1)
data=data.drop('new_emot',axis=1)
data=data.drop('new_skin',axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 0 to 1093
Columns: 654 entries, travel_history_to_country to yellowish
dtypes: float64(642), int64(8), object(4)
memory usage: 5.5+ MB


In [0]:

data=data.drop('Symptoms',axis=1)
data=data.drop('past_history_of_diseases',axis=1)
data=data.drop('emotional_problems',axis=1)
data=data.drop('change_in_skin_color',axis=1)

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 0 to 1093
Columns: 650 entries, travel_history_to_country to yellowish
dtypes: float64(642), int64(8)
memory usage: 5.4 MB


In [0]:
ytrain.dtype
from sklearn.model_selection import train_test_split
xtr,xte,ytr,yte=train_test_split(data,ytrain,test_size=0.33)

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
p_test3 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,200,250,350]}

tuning = GridSearchCV(estimator =GradientBoostingClassifier(max_depth=4, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10), 
            param_grid = p_test3, scoring='accuracy',n_jobs=4,iid=False, cv=5)
tuning.fit(data,ytrain)
print(tuning.grid_scores_, tuning.best_params_, tuning.best_score_)

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
gbrt=GradientBoostingClassifier(n_estimators=321,max_depth=4)
regr = cross_val_score(gbrt,data,ytrain,cv=4)
print(regr.mean())

0.9442661960910137
